### Advanced Exceptions 
- short intro to exceptions 
- Review of named attributes of exception objects 
- introduction to chained exceptions 
- analysis of traceback object of each exception 

### Exception - short intro 

An exception:
- Stops your program
- creates a special data called an exception (an object) 

We usually *raise* an exception 
- expects someone to notice and take care of it 
- program **forcibly terminated** and theres an error msg output 
- if **handled** properly, the program will resume and execution can continue 

Python tools allow us to **observe exceptions, identify them and handle them**

In [2]:
# Exception handling - try and except block 
try:
    print(int('a')) 
except ValueError as e_var: # variable set to this exception 
    print(e_var.args) # args present all the arguments in the variables exception instance 

("invalid literal for int() with base 10: 'a'",)


In [3]:
# importing a wrong module will bring up an importerror 
try:
    import asdasd
except ImportError as e:
    print(e.args, e.name, e.path) # args = arguments & name = name of the module & path = path to any file which triggered the exception 

("No module named 'asdasd'",) asdasd None


Something like `UnicodeError` has attributes such as 
- **encoding** -> name of the encoding raised 
- **reason** -> specific codec error 
- **object** -> obj codec was attempting to encode or decode 
- **start** -> first index of invalid data 
- **end** -> index after the last invalid data

In [4]:
try:
    b'\x80'.decode("utf-8")
except UnicodeError as e:
    print(e)
    print(e.encoding)
    print(e.reason)
    print(e.object)
    print(e.start)
    print(e.end)

'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
utf-8
invalid start byte
b'\x80'
0
1


### Chained Exceptions

Chaining Concept includes:
- the `__context__` attr for implicitly chained exception and `__cause__` attr for explicitly chained exception

In [11]:
a_list = ['First error', 'Second error']

try:
    print(a_list[3])
except Exception as e:
    try:
    # the following line is a developer mistake - they wanted to print progress as 1/10	but wrote 1/0
        print(1 / 0)
    except ZeroDivisionError as f:
        print('Inner exception (f):', f)
        print('Outer exception (e):', e)
        print('Outer exception referenced:', f.__context__) # the context is referencing the original exception object e 
        print('Is it the same object:', f.__context__ is e) # thats why this is true

Inner exception (f): division by zero
Outer exception (e): list index out of range
Outer exception referenced: list index out of range
Is it the same object: True


In [12]:
# Explicitly Chained Exceptions 
class RocketNetReadyErr(Exception):
    pass 

def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNetReadyErr('Crew not complete') from e # concerting an exception object to another exception object 
    
crew = ['John', 'Mary', 'Mike']
print('Final check procedure')

personnel_check()

Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike


RocketNetReadyErr: Crew not complete

In [14]:
try:
    personnel_check()
except RocketNetReadyErr as f: # excepting the Error Directly 
    # we're using the __cause__ attribute for explicitly seeing the exception 
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))
    print(f.__context__)


	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
General exception: "Crew not complete", caused by "list index out of range"
list index out of range


In [15]:
def fuel_check():
    try:
        print('Fuel tank is full in {}%'.format(100 / 0))
    except ZeroDivisionError as e:
        raise RocketNetReadyErr('Problem with fuel gauge') from e

crew = ['John', 'Mary', 'Mike']
fuel = 100
check_list = [personnel_check, fuel_check]
print('Final check procedure')

for check in check_list:
    try:
        check() # note that check var is referencing the function
    except RocketNetReadyErr as f:
        # because of polymorphism we could have multiple RocketNotReady Exceptions
        print('RocketNotReady exception: "{}", caused by "{}"'.format(f, f.__cause__))

Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
RocketNotReady exception: "Crew not complete", caused by "list index out of range"
RocketNotReady exception: "Problem with fuel gauge", caused by "division by zero"


In [19]:
# Looking at the traceback attribute
import traceback

crew = ['John', 'Mary', 'Mike']

print('Final check procedure')

try:
    personnel_check()
except RocketNetReadyErr as f:
    print(f.__traceback__) # we have the traceback details 
    print(type(f.__traceback__))
    
    print('\nTraceback Details')
    details = traceback.format_tb(f.__traceback__) # we throw in the traceback object into format_tb 
    print('\n'.join(details))



Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
<class 'traceback'>

Traceback Details
  File "C:\Users\justi\AppData\Local\Temp\ipykernel_5436\3615339854.py", line 9, in <module>
    personnel_check()

  File "C:\Users\justi\AppData\Local\Temp\ipykernel_5436\4059215711.py", line 12, in personnel_check
    raise RocketNetReadyErr('Crew not complete') from e
